In [ ]:
python src/tools/image_slicer.py --image_path 2025_08_05_PhotoRieboR4_g201b201078_f003_259.JPG --overlap_percentage 30 --destination_folder ./slices/ --slice-size 512

In [1]:
!nvidia-smi

Tue Sep 30 13:45:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.88                 Driver Version: 576.88         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090      WDDM  |   00000000:02:00.0  On |                  N/A |
|  0%   48C    P8             47W /  600W |    1946MiB /  32607MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
import torch
devices = []
print(f"CUDA доступна: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Название GPU: {torch.cuda.get_device_name(0)}")
    print(f"Количество GPU: {torch.cuda.device_count()}")
    print(f"Текущий GPU device: {torch.cuda.current_device()}")
    for i in range(torch.cuda.device_count()):
        devices.append(f'cuda:{i}')
    print(f"Доступные устройства: {devices}")
else:
    print("CUDA недоступна, используется CPU")
    device = 'cpu'


CUDA доступна: True
Название GPU: NVIDIA GeForce RTX 5090
Количество GPU: 2
Текущий GPU device: 0
Доступные устройства: ['cuda:0', 'cuda:1']


In [5]:
image_path = "2025_08_05_PhotoRieboR4_g201b201078_f003_259.JPG"

In [47]:
slice_path = r"slices\2025_08_05_PhotoRieboR4_g201b201078_f003_259\2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_007_007_3227_3227.png"

In [48]:
predictions = model.predict(slice_path, conf=0.1, device=device)
print(f"Количество детекций: {len(predictions[0].boxes.conf)}")
print(f"Используемое устройство: {device}")
print(predictions[0].boxes.xyxy)
list_of_boxes = predictions[0].boxes.xyxy.tolist()


image 1/1 c:\Users\sduai\Desktop\Batyrbayev_Adilet\qCloudy\Projects\PoC\Saiga\slices\2025_08_05_PhotoRieboR4_g201b201078_f003_259\2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_007_007_3227_3227.png: 640x640 6 birds, 28.6ms
Speed: 3.6ms preprocess, 28.6ms inference, 5.2ms postprocess per image at shape (1, 3, 640, 640)
Количество детекций: 6
Используемое устройство: cuda
tensor([[3.3217e+02, 1.2781e-01, 3.6921e+02, 4.3585e+01],
        [3.6726e+02, 2.9117e+02, 4.0825e+02, 3.1028e+02],
        [3.2111e+02, 4.2453e+02, 3.5456e+02, 4.5206e+02],
        [2.8269e+02, 4.0887e+02, 3.1934e+02, 4.4877e+02],
        [4.0217e+02, 3.7962e+02, 4.2558e+02, 4.0467e+02],
        [4.7605e+02, 4.0503e+02, 5.1181e+02, 4.3833e+02]], device='cuda:0')


In [17]:
predictions[0].save("predicted_image.png")

'predicted_image.png'

In [ ]:
import os
from src.tools.image_slicer import create_image_slices
images_folder = "dataset/0.0.1/АФС для обработки ИИ"
image_paths = [f"{images_folder}/{image_path}" for image_path in os.listdir(images_folder)]
for image_path in image_paths:
    create_image_slices(
        image_path=image_path, 
        overlap_percentage=10, 
        destination_folder="slices",
        slice_size=512
    )

In [6]:
import torch
print(f"CUDA доступна: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Название GPU: {torch.cuda.get_device_name(0)}")
    print(f"Количество GPU: {torch.cuda.device_count()}")
    print(f"Текущий GPU device: {torch.cuda.current_device()}")
    device = 'cuda'
    print(f"Доступные устройства: {device}")
else:
    print("CUDA недоступна, используется CPU")
    device = 'cpu'

CUDA доступна: True
Название GPU: NVIDIA GeForce RTX 5090
Количество GPU: 2
Текущий GPU device: 0
Доступные устройства: cuda


In [7]:
model_name = 'best_4'
model = YOLO(f'{model_name}.pt')
model.to(device)
print(f"Модель загружена на: {device}")

Модель загружена на: cuda


In [8]:
images_folder = "slices"

In [ ]:
import os
import time

output_folder = f"predicted_images-{model_name}"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

print(f"Обработка изображений на устройстве: {device}")
start_time = time.time()
processed_count = 0
total_detections = 0
boxes_list = []
for folder in os.listdir(images_folder):
    folder_path = f"{images_folder}/{folder}"
    if os.path.isdir(folder_path):
        for image_path in os.listdir(folder_path):
            full_image_path = f"{folder_path}/{image_path}"
            predictions = model.predict(full_image_path, conf=0.1, device=device, verbose=False)
            
            if len(predictions[0].boxes.conf) > 0:
                boxes_list.append({'source_image_path': full_image_path, 'coordinates': predictions[0].boxes.xyxy.tolist()})
                if not os.path.exists(f"{output_folder}/{folder}"):
                    os.makedirs(f"{output_folder}/{folder}")
                predictions[0].save(f"{output_folder}/{folder}/{image_path}")
                processed_count += 1
                total_detections += len(predictions[0].boxes.conf)
                if processed_count % 10 == 0:  
                    print(f"Обработано: {processed_count} изображений с детекциями")

end_time = time.time()
print(f"\nГотово! Обработано {processed_count} изображений с детекциями")
print(f"Общее количество детекций: {total_detections}")
print(f"Время обработки: {end_time - start_time:.2f} секунд")
print(f"Устройство: {device}")


In [31]:
import torch
from ultralytics import YOLO
import os
import time
import sys
if 'src.config' in sys.modules:
    del sys.modules['src.config']
from src.config import MODEL_NAME, SLICES_FOLDER, OUTPUT_FOLDER_PREFIX
print(MODEL_NAME, SLICES_FOLDER, OUTPUT_FOLDER_PREFIX)

best_4 slices predicted_images


In [ ]:

def get_devices():
    devices = []
    print(f"CUDA доступна: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"Название GPU: {torch.cuda.get_device_name(0)}")
        print(f"Количество GPU: {torch.cuda.device_count()}")
        print(f"Текущий GPU device: {torch.cuda.current_device()}")
        for i in range(torch.cuda.device_count()):
            devices.append(f'cuda:{i}')
        print(f"Доступные устройства: {devices}")
    else:
        print("CUDA недоступна, используется CPU")
        devices.append('cpu')
    return devices

def get_models(model_name, devices):
    models = []
    for device in devices:
        models.append(YOLO(f'{model_name}.pt'))
        models[-1].to(device)
        print(f"Модель загружена на: {device}")
    return models

def create_destination_folder(output_folder_prefix, model_name):
    output_folder = f"{output_folder_prefix}-{model_name}"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    return output_folder

def copyfile(source_path, destination_path):
    with open(source_path, 'rb') as src, open(destination_path, 'wb') as dst:
        dst.write(src.read())

def create_classes_file(output_folder):
    with open(f"{output_folder}/classes.txt", "w") as f:
        f.write("saiga\n")

def create_annotation_file(image_path, boxes, output_folder, image_size=512):
    if not os.path.exists(f"{output_folder}/classes.txt"):
        create_classes_file(output_folder)
    
    filename = image_path.split('/')[-1].split('.')[0]
    with open(f"{output_folder}/{filename}.txt", "w") as f:  # "w" вместо "a"
        for box in boxes:
            x_center = (box[0] + box[2]) / 2 / image_size
            y_center = (box[1] + box[3]) / 2 / image_size
            width = (box[2] - box[0]) / image_size
            height = (box[3] - box[1]) / image_size
            f.write(f"0 {x_center} {y_center} {width} {height}\n")

def process_images(model, images_folder, output_folder, device):
    print(f"Обработка изображений на устройстве: {device}")
    start_time = time.time()
    processed_count = 0
    total_detections = 0
    boxes_list = []
    for folder in os.listdir(images_folder):
        folder_path = f"{images_folder}/{folder}"
        if os.path.isdir(folder_path):
            for image_path in os.listdir(folder_path):
                full_image_path = f"{folder_path}/{image_path}"
                predictions = model.predict(full_image_path, conf=0.1, device=device, verbose=False)
                
                if len(predictions[0].boxes.conf) > 0:
                    boxes_data = {'source_image_path': full_image_path, 'coordinates': predictions[0].boxes.xyxy.tolist()}
                    boxes_list.append(boxes_data)
                    if not os.path.exists(f"{output_folder}/{folder}"):
                        os.makedirs(f"{output_folder}/{folder}")
                    final_image_path = f"{output_folder}/{folder}/{image_path}"
                    copyfile(full_image_path, final_image_path)
                    # predictions[0].save(final_image_path)
                    create_annotation_file(full_image_path, predictions[0].boxes.xyxy.tolist(), f"{output_folder}/{folder}")                  
                    processed_count += 1
                    total_detections += len(predictions[0].boxes.conf)
                    if processed_count % 10 == 0:  
                        print(f"Обработано: {processed_count} изображений с детекциями")

    end_time = time.time()
    print(f"\nГотово! Обработано {processed_count} изображений с детекциями")
    print(f"Общее количество детекций: {total_detections}")
    print(f"Время обработки: {end_time - start_time:.2f} секунд")
    print(f"Устройство: {device}")
    return boxes_list

devices = get_devices()
models = get_models(MODEL_NAME, devices)
output_folder = create_destination_folder(OUTPUT_FOLDER_PREFIX, MODEL_NAME)
boxes_list = process_images(models[0], SLICES_FOLDER, output_folder, devices[0])


In [ ]:
boxes_list

In [ ]:
print("boxes_list", len(boxes_list))
for boxes in boxes_list:
    x_min = int(boxes['source_image_path'].split('/')[-1].split('_')[-2].split('.')[0])
    y_min = int(boxes['source_image_path'].split('/')[-1].split('_')[-1].split('.')[0])

    for box in boxes['coordinates']:
        box[0] = box[0] + x_min
        box[1] = box[1] + y_min
        box[2] = box[2] + x_min
        box[3] = box[3] + y_min

boxes_list

In [ ]:
# let's count outliers for each image
import numpy as np
print("boxes_list", len(boxes_list))

filtered_boxes_from_outliers = []
for boxes in boxes_list:
    areas = []
    for box in boxes['coordinates']:
        box_area = (box[2] - box[0]) * (box[3] - box[1])
        areas.append(box_area)
    mean_of_areas = sum(areas) / len(areas)

    q1 = np.percentile(areas, 25)
    q3 = np.percentile(areas, 75)
    iqr = q3 - q1
    lower_bound = q1 - 3 * iqr
    upper_bound = q3 + 3 * iqr
    filtered_coordinates_from_outliers = []
    for box in boxes['coordinates']:
        box_area = (box[2] - box[0]) * (box[3] - box[1])
        if box_area > lower_bound and box_area < upper_bound:
            filtered_coordinates_from_outliers.append(box)
    
    print("sum of areas", sum(areas))
    print("mean of areas", mean_of_areas)
    print("lower_bound", lower_bound)
    print("upper_bound", upper_bound)
    print("iqr", iqr)
    print("amount of filtered_coordinates_from_outliers", len(filtered_coordinates_from_outliers))
    if len(filtered_coordinates_from_outliers) > 0:
        filtered_boxes_from_outliers.append({'source_image_path': boxes['source_image_path'], 'coordinates': filtered_coordinates_from_outliers})

print("amount of filtered_boxes_from_outliers", len(filtered_boxes_from_outliers))


In [ ]:
# Импорты уже выполнены в предыдущей ячейке
def calculate_iou(box1, box2):
    # box = [x1, y1, x2, y2]
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    print("intersection_area", intersection_area)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - intersection_area
    print("union_area", union_area)

    if union_area == 0:
        return 0.0

    return intersection_area / union_area

threshold = 0.5

all_boxes = []
for box_group in boxes_list:
    source_path = box_group['source_image_path']
    for coord in box_group['coordinates']:
        all_boxes.append({
            'coordinates': coord,
            'source': source_path
        })

with_duplicates = len(all_boxes)
print(f"Всего боксов для сравнения: {with_duplicates}")
count = 0
for i in range(len(all_boxes)):
    for j in range(i + 1, len(all_boxes)):
        iou = calculate_iou(all_boxes[i]['coordinates'], all_boxes[j]['coordinates'])
        # print("iou", iou)
        if iou > threshold:
            print(f"Боксы с IoU > {threshold}:")
            print(f"Бокс 1: {all_boxes[i]['coordinates']} (из {all_boxes[i]['source']})")
            print(f"Бокс 2: {all_boxes[j]['coordinates']} (из {all_boxes[j]['source']})")         
            print(f"IoU: {iou:.4f}")
            print("---")
            count += 1

print(f"Количество боксов с IoU > {threshold}: {count}")
without_duplicates = len(all_boxes) - count
print(f"Количество боксов без дубликатов: {without_duplicates}")

In [ ]:
import numpy as np

sum_of_areas = 0
areas = []
for box in all_boxes:
    box_area = (box['coordinates'][2] - box['coordinates'][0]) * (box['coordinates'][3] - box['coordinates'][1])
    sum_of_areas += box_area
    areas.append(box_area)
print("sum of areas in areas", sum(areas))
print("mean of areas in areas", sum(areas) / len(areas))
print("sum_of_areas", sum_of_areas)
mean_of_areas = sum_of_areas / len(all_boxes)
print("mean_of_areas", mean_of_areas)
iqr = np.percentile(areas, 75) - np.percentile(areas, 25)
print("iqr", iqr)
without_duplicates_and_outliers = without_duplicates
outliers_count = 0
for box in all_boxes:
    # for each 'source' it has to count outliers

    box_area = (box['coordinates'][2] - box['coordinates'][0]) * (box['coordinates'][3] - box['coordinates'][1])
    if box_area > mean_of_areas + 3 * iqr:
        print("plus outliersbox_area", box_area)
        print("box", box)
        print("---")
        without_duplicates_and_outliers += 1
        # outliers_count += 1
    if box_area < mean_of_areas - 3 * iqr:
        print("minus outliers box_area", box_area)
        print("box", box)
        print("---")
        without_duplicates_and_outliers -= 1
        outliers_count += 1
print("outliers_count", outliers_count)

In [15]:
without_duplicates_and_outliers

47